In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Import Graphs

In [4]:
import os
import networkx as nx
import pickle

graph_directory = "./graphs/er_700-800/"

datasets = []

for filename in os.listdir(graph_directory):
    if filename.endswith(".gpickle"):
        print("ER 700-800 Graph ", os.path.join(graph_directory, filename), "is being imported ...")
        with open(os.path.join(graph_directory, filename), 'rb') as f:
            G = pickle.load(f)
            datasets.append(
            {
                "name": filename[:-8],
                "graph": nx.relabel.convert_node_labels_to_integers(
                    G, first_label=0
                ),
            })

ER 700-800 Graph  ./graphs/er_700-800/ER_700_800_0.15_18.gpickle is being imported ...
ER 700-800 Graph  ./graphs/er_700-800/ER_700_800_0.15_19.gpickle is being imported ...
ER 700-800 Graph  ./graphs/er_700-800/ER_700_800_0.15_120.gpickle is being imported ...
ER 700-800 Graph  ./graphs/er_700-800/ER_700_800_0.15_10.gpickle is being imported ...
ER 700-800 Graph  ./graphs/er_700-800/ER_700_800_0.15_12.gpickle is being imported ...
ER 700-800 Graph  ./graphs/er_700-800/ER_700_800_0.15_122.gpickle is being imported ...
ER 700-800 Graph  ./graphs/er_700-800/ER_700_800_0.15_94.gpickle is being imported ...
ER 700-800 Graph  ./graphs/er_700-800/ER_700_800_0.15_15.gpickle is being imported ...
ER 700-800 Graph  ./graphs/er_700-800/ER_700_800_0.15_53.gpickle is being imported ...
ER 700-800 Graph  ./graphs/er_700-800/ER_700_800_0.15_97.gpickle is being imported ...
ER 700-800 Graph  ./graphs/er_700-800/ER_700_800_0.15_41.gpickle is being imported ...
ER 700-800 Graph  ./graphs/er_700-800/ER_

## Define Solver Parameters

In [5]:
from solvers.dNNMIS_GPU_TAU import DNNMIS as V2
from solvers.KaMIS import ReduMIS
from solvers.ILPMIS import ILPMIS
from solvers.Quadratic import Quadratic

solvers = [
    # {
    #     "name": "dNN V2 GPU",
    #     "class": V2,
    #     "params": {
    #         "learning_rate": 0.05,
    #         "selection_criteria": 0.45,
    #         "max_steps": 20000
    #     },
    # },
    {
        "name": "Quadratic GPU",
        "class": Quadratic,
        "params": {
            "learning_rate": 0.5,
            "max_steps": 75000
        },
    },
    # {
    #     "name": "dNN",
    #     "class": DNNMIS,
    #     "params": {
    #         "learning_rate": 0.001,
    #         "selection_criteria": 0.8,
    #         "max_steps": 25000,
    #     },
    # },
    #     {
    #     "name": "dNN w/SG5k",
    #     "class": DNNMIS,
    #     "params": {
    #         "learning_rate": 0.001,
    #         "selection_criteria": 0.65,
    #         "max_steps": 80000,
    #         "max_subgraph_steps": 5000,
    #     },
    # },
    # {"name": "ReduMIS", "class": ReduMIS, "params": {"seed": 13}},
    # {"name": "ILP", "class": ILPMIS, "params": {"time_limit": 300}}
]

## Optional: Run New Benchmark

In [6]:
from copy import deepcopy


def benchmark(datasets, solvers):
    solutions = []

    stage = 0
    stages = len(solvers) * len(datasets)

    for solver in solvers:
        for dataset in datasets:
            solver_instance = solver["class"](dataset["graph"], solver["params"])
            solver_instance.solve()
            solution = {
                "solution_method": solver["name"],
                "dataset_name": dataset["name"],
                "data": deepcopy(solver_instance.solution),
                "time_taken": deepcopy(solver_instance.solution_time),
            }
            solutions.append(solution)
            del solver_instance
            stage += 1
            print(f"Completed {stage} / {stages}")

    return solutions

solutions = benchmark(datasets, solvers)

using device:  cuda:0


/home/cledenmat/github.com/ledenmat/DatalessNN/solvers/Quadratic.py:18: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3614.)
  loss = -Matrix_X.sum() + (gamma/2) * (Matrix_X.T @ (adjacency_matrix_tensor) @ Matrix_X) - (beta/2) * (Matrix_X.T @ (adjacency_matrix_tensor_comp) @ Matrix_X)


Step 1000/75000, IS: tensor([  3,  11,  69, 100, 112, 140, 163, 169, 190, 195, 200, 211, 218, 241,
        246, 322, 364, 372, 387, 449, 468, 475, 486, 497, 501, 536, 581, 613,
        630, 631, 670, 691, 693, 694, 717], device='cuda:0'), lr: 0.5, Loss: 0.0
Step 2000/75000, IS: tensor([  5,  16,  80, 140, 157, 160, 167, 169, 172, 190, 194, 195, 219, 246,
        247, 322, 372, 388, 393, 399, 468, 486, 492, 497, 501, 531, 534, 613,
        630, 658, 694, 695, 701, 713, 727, 744, 749, 782], device='cuda:0'), lr: 0.5, Loss: 0.0
Step 3000/75000, IS: tensor([  5,  16,  80, 140, 157, 160, 167, 169, 172, 190, 194, 195, 219, 246,
        247, 322, 372, 388, 393, 399, 468, 486, 492, 497, 501, 531, 534, 613,
        630, 658, 694, 695, 701, 713, 727, 744, 749, 782], device='cuda:0'), lr: 0.5, Loss: 0.0
Step 4000/75000, IS: tensor([  3,   4,  16,  50,  68,  73,  75,  80, 112, 140, 163, 190, 195, 241,
        246, 314, 316, 322, 356, 364, 372, 382, 387, 428, 468, 492, 496, 497,
        515, 536, 5

## Generate Results Table

In [7]:
import pandas
import matplotlib.pyplot as plt
from itertools import combinations

dataset_index = {k: v for v, k in enumerate([dataset["name"] for dataset in datasets])}
datasets_solutions = [[] for i in range(len(datasets))]

def MIS_checker(MIS_list, G):
    pairs = list(combinations(MIS_list, 2))
    IS_CHECKER = True
    if len(MIS_list) > 1:
        for pair in pairs:
            if (pair[0], pair[1]) in G.edges or (pair[1], pair[0]) in G.edges:
                IS_CHECKER = False
                break
    return IS_CHECKER

table_data = []

for solution in solutions:
    dsi = dataset_index[solution["dataset_name"]]
    datasets_solutions[dsi].append(solution)

i = 0
for dataset_solutions in datasets_solutions:
    # IS CHECK
    is_check = []
    for solution in dataset_solutions:
        IS_set = solution["data"]["graph_mask"]
        # print(IS_set)
        # indices = [i for i, x in enumerate(IS_set) if x == 1]
        # print(indices)
        # subgraph = datasets[dataset_index[solution["dataset_name"]]]["graph"].subgraph(indices)
        # subgraph = nx.Graph(subgraph)
        # while len(subgraph) > 0:
        #     degrees = dict(subgraph.degree())
        #     max_degree_nodes = [node for node, degree in degrees.items() if degree == max(degrees.values())]
            
        #     if len(max_degree_nodes) == 0 or subgraph.degree(max_degree_nodes[0]) == 0:
        #         break  # No more nodes to remove or all remaining nodes have degree 0

        #     subgraph.remove_node(max_degree_nodes[0])
        # nodes = list(subgraph)
        # nodes.sort()
        # print(nodes)
        # subgraph = datasets[dataset_index[solution["dataset_name"]]]["graph"].subgraph(
        #     IS_set
        # )
        # if len(subgraph.edges) > 0:
        #     plt.figure(i)
        #     plt.title(subgraph.edges)
        #     i += 1
        #     is_check.append(False)
        #     print(
        #         f"Non IS found using {solution['solution_method']} on {solution['dataset_name']}"
        #     )
        is_check.append(True)


    table_row = [dataset_solutions[0]['dataset_name']]

    table_row.extend([solution["data"]["size"] for solution in dataset_solutions])
    table_row.extend([solution["time_taken"] for solution in dataset_solutions])
    table_row.extend(is_check)

    table_data.append(table_row)

table_headers = ["Dataset Name"]

table_headers.extend([solver["name"] + " Solution Size" for solver in solvers])
table_headers.extend([solver["name"] + " Solution Time" for solver in solvers])
table_headers.extend([solver["name"] + " Solution IS" for solver in solvers])

table = pandas.DataFrame(table_data, columns=table_headers)
table
table.to_csv("results.csv")

## Optional: Save Solutions to Pickle

In [8]:
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

def read_object(filename):
    with open(filename, 'rb') as outp:  # Overwrites any existing file.
        return pickle.load(outp)

# solutions = save_object(solutions, "solutions.pkl")